In [1]:
import pandas as pd
import numpy as np
import review_preprocess
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout,Embedding,SpatialDropout1D,LSTM
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
train_file_name = 'drugsComTrain_raw.csv'
test_file_name = 'drugsComTest_raw.csv'
file_path = './data/'

df_test = pd.read_csv(file_path+test_file_name)
df_train = pd.read_csv(file_path+train_file_name)

In [3]:
df_processed_train = review_preprocess.clean_reviews(df_train,is_lstm=True)

Review cleanup Completed...aracters from review column--> 100.0 percentage complete
Removing row with nan values
Percentage of nan rows in dataset--> 0.56 %
Removed 899 rows with na values


In [4]:
df_processed_test = review_preprocess.clean_reviews(df_test,is_lstm=True)

Review cleanup Completed...aracters from review column--> 100.0 percentage complete
Removing row with nan values
Percentage of nan rows in dataset--> 0.55 %
Removed 295 rows with na values


In [5]:
m = 0
for t in df_processed_train.review:
    a = t.split()
    if(len(a)>m):
        m=len(a)
print("Max words in a review inside train set is ",m)
m = 0
for t in df_processed_test.review:
    a = t.split()
    if(len(a)>m):
        m=len(a)
print("Max words in a review inside test set is ",m)

Max words in a review inside train set is  945
Max words in a review inside test set is  540


In [6]:
# number of unique words in review are 
# len(.values.split()))
all_review = ' '.join(df_processed_train['review'])
print("Total words combined in all train review ",len(all_review))
print("Total unique words in all train review ",len(set(all_review.split(" "))))

Total words combined in all train review  45482391
Total unique words in all train review  89615


In [13]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=2500)#max_features=4000
x_tfidf = tfidf_vectorizer.fit_transform(df_processed_train.review)

hash_vectorizer = HashingVectorizer(n_features=1000)#n_features=100
x_hash = hash_vectorizer.fit_transform(df_processed_train.review)

In [14]:
print("TFIDF  ",x_tfidf.shape)
print("hash  ",x_hash.shape)

TFIDF   (160398, 2500)
hash   (160398, 1000)


In [15]:
df_processed_train.rating
y_train = pd.get_dummies(df_processed_train.rating).values

y_test = pd.get_dummies(df_processed_test.rating).values

In [16]:
MAX_NB_WORDS = 500
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 200
# This is fixed.
EMBEDDING_DIM = 128

model = Sequential()
model.add(Embedding(2500, EMBEDDING_DIM, input_length=x_tfidf.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64


In [17]:
history = model.fit(x_tfidf, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 144358 samples, validate on 16040 samples
Epoch 1/5
    64/144358 [..............................] - ETA: 25:15:38 - loss: 2.2908 - accuracy: 0.1719

KeyboardInterrupt: 